In [1]:
import igraph as ig  
import pandas as pd
import json
import random

In [2]:
nodes_path = "../file_data/fb-pages-food.nodes"
edges_path = "../file_data/fb-pages-food.edges"

nodes_df = pd.read_csv(
    nodes_path, sep=",", header=None, names=["node", "name", "new_id"], skiprows=1
)
edges_df = pd.read_csv(edges_path, sep=",", header=None, names=["source", "target"])

nodes_df["new_id"] = nodes_df["new_id"].astype(str)
edges_df["source"] = edges_df["source"].astype(str)
edges_df["target"] = edges_df["target"].astype(str)

In [3]:
g = ig.Graph()
g.add_vertices(nodes_df["new_id"].tolist())
g.add_edges(edges_df[["source", "target"]].values.tolist())
g.vs["name"] = nodes_df["name"].tolist()

In [4]:
communities = g.community_edge_betweenness()
communities = communities.as_clustering()

In [5]:
def generate_community_names(num_cliques):
    prefixes = [
        "Neo",
        "Alt",
        "Hyper",
        "Meta",
        "Quantum",
        "Terra",
        "Cyber",
        "Omega",
        "Solar",
        "Astra",
    ]
    roots = [
        "Gen",
        "Net",
        "Sphere",
        "Cluster",
        "Link",
        "Wave",
        "Flow",
        "Pulse",
        "Code",
        "Echo",
    ]
    suffixes = ["ium", "on", "os", "ix", "ia", "or", "an", "is", "us", "er"]

    community_names = set()

    while len(community_names) < num_cliques:
        name = (
            f"{random.choice(prefixes)}{random.choice(roots)}{random.choice(suffixes)}"
        )
        name += f" {random.randint(1000, 9999)}"
        if name not in community_names:
            community_names.add(name)
    print("nomi generati")
    return list(community_names)

In [6]:
def assign_community_names_to_graph(graph, communities):
    num_communities = len(communities)
    community_names = generate_community_names(num_communities)
    updated_communities = []

    if "aggregation" not in graph.vs.attributes():
        graph.vs["aggregation"] = [[] for _ in range(graph.vcount())]

    if "community_name" not in graph.vs.attributes():
        graph.vs["community_name"] = [None for _ in range(graph.vcount())]

    for i, community in enumerate(communities):
        community_name = community_names[i]
        updated_communities.append({"name": f"cliques {community_name}", "nodes": community})
        for node in community:
            graph.vs[node]["aggregation"].append(f"community {community_name}")
            if graph.vs[node]["community_name"] is None:
                graph.vs[node]["community_name"] = community_name
    if "community_name" not in graph.es.attributes():
        graph.es["community_name"] = [None] * graph.ecount()
    for edge in graph.es:
        source_node = edge.source  
        graph.es[edge.index]["community_name"] = graph.vs[source_node]["community_name"]

    return updated_communities

In [7]:
updated_communities = assign_community_names_to_graph(g, communities)
for node in g.vs:
    print(f"Node {node.index}: Community Name -> {node['community_name']}")

nomi generati
Node 0: Community Name -> AstraLinkor 5650
Node 1: Community Name -> AstraLinkor 5650
Node 2: Community Name -> AstraLinkor 5650
Node 3: Community Name -> QuantumGenix 3514
Node 4: Community Name -> AstraLinkor 5650
Node 5: Community Name -> AstraLinkor 5650
Node 6: Community Name -> QuantumGenix 3514
Node 7: Community Name -> AstraLinkor 5650
Node 8: Community Name -> QuantumCodeium 3186
Node 9: Community Name -> AstraLinkor 5650
Node 10: Community Name -> OmegaGenos 2464
Node 11: Community Name -> CyberLinkan 9277
Node 12: Community Name -> TerraSphereix 1504
Node 13: Community Name -> QuantumGenix 3514
Node 14: Community Name -> NeoClusteran 8337
Node 15: Community Name -> TerraEchoos 6531
Node 16: Community Name -> AstraLinkor 5650
Node 17: Community Name -> AstraLinkor 5650
Node 18: Community Name -> AstraLinkor 5650
Node 19: Community Name -> HyperNetix 8184
Node 20: Community Name -> NeoLinkis 5587
Node 21: Community Name -> HyperLinker 1196
Node 22: Community Name

In [9]:
output_data = {
    "graph": {
        "nodes": [
            {
                "id": node.index,
                "name": node["name"],
                "community_name": node["community_name"],
                "aggregation": node["aggregation"],
            }
            for node in g.vs
        ],
        "edges": [{"source": edge.source, "target": edge.target, "community_name": edge["community_name"]} for edge in g.es],
    },
    "communities": [community for community in updated_communities],
}

with open("../graph_data/graph_data_communities.json", "w") as f:
    json.dump(output_data, f, indent=4)